# Access OCR Output Data

This guide shows how to access OCR's wind-adjusted fire risk output data using [Icechunk](https://icechunk.io/en/latest/), a versioned data format for cloud-native geospatial data.

## Prerequisites

- Python environment with `xarray` and `icechunk` installed

## What you'll learn

- How to connect to OCR's production Icechunk repository
- How to open and inspect the fire risk dataset
- How to work with specific variables and spatial subsets

## Step 1: Import required libraries

In [1]:
import icechunk
import xarray as xr

## Step 2: Connect to the Icechunk repository

OCR's production fire risk data is stored in an Icechunk repository on S3. We'll connect to version `v0.5.0` of the wind-adjusted fire risk output. For valid versions, check out the [OCR's GitHub release page](https://github.com/carbonplan/ocr/releases).

In [2]:
# Configure S3 storage for the Icechunk repository
version = 'v0.8.0'
storage = icechunk.s3_storage(
    bucket='carbonplan-ocr',
    prefix=f'output/fire-risk/tensor/production/{version}/ocr.icechunk',
    anonymous=True,
)

# Open the repository
repo = icechunk.Repository.open(storage)

# Create a read-only session on the main branch
session = repo.readonly_session('main')

## Step 3: Open the dataset with xarray


In [3]:
# Open the dataset
ds = xr.open_dataset(session.store, engine='zarr', chunks={})
ds

<xarray.Dataset> Size: 652GB
Dimensions:                     (latitude: 97579, longitude: 208881)
Coordinates:
  * latitude                    (latitude) float64 781kB 52.48 52.48 ... 22.43
  * longitude                   (longitude) float64 2MB -128.4 -128.4 ... -64.05
Data variables:
    burn_probability_usfs_2011  (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    USFS_RPS                    (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    burn_probability_2047       (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    conditional_risk_usfs       (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    burn_probability_usfs_2047  (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    wind_risk_2011              (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    burn_probability_2011       (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    wind_risk_2047              (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>

## Step 4: Explore the data variables

The dataset contains wind-adjusted fire risk metrics. Let's examine the available variables:

In [4]:
# List all data variables
print('Data variables:')
for var in ds.data_vars:
    print(f'  - {var}: {ds[var].attrs.get("long_name", "No description")}')

Data variables:
  - burn_probability_usfs_2011: No description
  - USFS_RPS: No description
  - burn_probability_2047: No description
  - conditional_risk_usfs: No description
  - burn_probability_usfs_2047: No description
  - wind_risk_2011: No description
  - burn_probability_2011: No description
  - wind_risk_2047: No description


## Step 5: Select a spatial subset

Extract data for a specific geographic region using coordinate slicing:

In [5]:
# Example: Select data for California region
california_subset = ds.sel(
    latitude=slice(42, 32),  # Southern to Northern California
    longitude=slice(-125, -114),  # Western to Eastern California
)

california_subset

<xarray.Dataset> Size: 37GB
Dimensions:                     (latitude: 32468, longitude: 35715)
Coordinates:
  * latitude                    (latitude) float64 260kB 42.0 42.0 ... 32.0 32.0
  * longitude                   (longitude) float64 286kB -125.0 ... -114.0
Data variables:
    burn_probability_usfs_2011  (latitude, longitude) float32 5GB dask.array<chunksize=(1967, 2501), meta=np.ndarray>
    USFS_RPS                    (latitude, longitude) float32 5GB dask.array<chunksize=(1967, 2501), meta=np.ndarray>
    burn_probability_2047       (latitude, longitude) float32 5GB dask.array<chunksize=(1967, 2501), meta=np.ndarray>
    conditional_risk_usfs       (latitude, longitude) float32 5GB dask.array<chunksize=(1967, 2501), meta=np.ndarray>
    burn_probability_usfs_2047  (latitude, longitude) float32 5GB dask.array<chunksize=(1967, 2501), meta=np.ndarray>
    wind_risk_2011              (latitude, longitude) float32 5GB dask.array<chunksize=(1967, 2501), meta=np.ndarray>
    burn_probability_2011       (latitude, longitude) float32 5GB dask.array<chunksize=(1967, 2501), meta=np.ndarray>
    wind_risk_2047              (latitude, longitude) float32 5GB dask.array<chunksize=(1967, 2501), meta=np.ndarray>